In [1]:
import argparse
import torch
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import BertConfig, BertForSequenceClassification, AdamW, get_scheduler
# from datasets import load_metric
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.datasets import load_svmlight_file

In [2]:
df = pd.read_csv("annotated_corpus.csv")

In [3]:
hall = []
for j in df.values:
#     print(j[4])
    if j[4] == 0 and j[5] == 0 and j[6] == 0 and j[7] == 0 and j[8] == 0:
        hall.append(0)
    else:
        hall.append(1)

In [4]:
df['hallucinate'] = hall
df['src_mt'] = df['src'].astype(str) + df['mt'].astype(str)

In [15]:
#take extra non hallucinated examples (total non_hallucinated - hallucinated) examples and perturb
df1_ = df.loc[df['hallucinate'] == 0].sample(1175)

In [72]:
df11_ = df.loc[~df.index.isin(df1_.index)]

In [19]:
df1_misspell = df1_.sample(588)
df1_random_token_src = df1_.loc[~df1_.index.isin(df1_misspell.index)]

In [20]:
print(df1_misspell.shape)
print(df1_random_token_src.shape)

(588, 11)
(587, 11)


### Misspelling 

In [63]:
#misspelt dataframe 

import random

misspelt = []
word_po = []

for i in df1_misspell.mt.values:
    words = i.split()
    word_pos = random.randint(0, len(words)-1) # randomly choose a word 
    word_po.append(word_pos)
    w = words[word_pos] # get the word
    i = random.randint(0, len(w)-1) # randomly choose the character to delete 
#     misspelt.append(w[:i] + w[i+1:])
    words[word_pos] = w[:i] + w[i+1:] # insert the deleted character word to the pos
    misspelt.append(" ".join(words))

In [67]:
df1_misspell['mt'] = misspelt 
df1_misspell['src_mt'] = df1_misspell['src'].astype(str) + df1_misspell['mt'].astype(str)

### Random token insertion 

In [88]:
# find top tokens in the text

from collections import Counter

lis = []

for i in df.mt.values:
    lis.append(i.split())

def flatten(input):
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list

l = flatten(lis)

In [102]:
587/2

293.5

In [119]:
most = pd.Series(l).value_counts()[:293].index.tolist()
least = pd.Series(l).value_counts()[-294:].index.tolist()

In [136]:
ins = []


for (i,j) in zip(df1_random_token_src.mt[0:len(most)].values, most):
    k = j + " " + i
    ins.append(k)


for (i,j) in zip(df1_random_token_src.mt[len(most):].values, least):
    k = j + " " + i
    ins.append(k)

In [140]:
df1_random_token_src['mt'] = ins 
df1_random_token_src['src_mt'] = df1_random_token_src['src'].astype(str) + df1_random_token_src['mt'].astype(str)

/raid/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/raid/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [143]:
df1_random_token_src['hallucinate'] = 1
df1_misspell['hallucinate'] = 1 

/raid/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [144]:
df = pd.concat([df11_, df1_misspell, df1_random_token_src], ignore_index=True)

In [146]:
df.hallucinate.value_counts()

0    1708
1    1707
Name: hallucinate, dtype: int64

In [147]:
df.to_csv("Hallucination/annotated_corpus_perturbed.csv")